In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
! pip install focal_loss 
import focal_loss 
from focal_loss import SparseCategoricalFocalLoss

In [5]:
! pip install transformers

import tensorflow as tf
import pandas as pd
import numpy as np
import os
import transformers
from transformers import BertTokenizer
from transformers import TFBertForSequenceClassification
from transformers import TFBertModel,  BertConfig, BertTokenizerFast
from sklearn.preprocessing import MultiLabelBinarizer
from itertools import repeat
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from transformers import TFBertModel,  BertConfig, BertTokenizerFast
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, Input, Dropout, Activation #, LSTM
from  tensorflow.compat.v1.keras.layers import CuDNNLSTM 
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Embedding, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy,BinaryCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy,BinaryAccuracy
from tensorflow.keras.utils import to_categorical
#from tensorflow_addons.losses import SigmoidFocalCrossEntropy
# And pandas for data import + sklearn because you allways need sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import TFBertForTokenClassification


labels_dict = {
'O'     : 0,    
'B-art' :   1,  
'B-eve' :   2,
'B-geo' : 3,
'B-gpe' : 4,
'B-nat' :   5,
'B-org' : 6,
'B-per' : 7,
'B-tim' : 8,
'I-art' :   9,
'I-eve' :   10,
'I-geo' :  11,
'I-gpe' :   12,
'I-nat' :    13,
'I-org' :  14,
'I-per' : 15,
'I-tim' :  16
}

data = pd .read_csv('/content/drive/MyDrive/NER/processed_ner_data_2.csv')
#data1 = data.iloc[1000,:].copy()
print(data.shape)
ner =  pd.DataFrame({
    'sentences' : data['sentences'],
    'tag_str' : data['tag_str']
})

ner_tag_list = list(map(lambda x: x.split(',')[1:], ner['tag_str'] ))
ner['ner_tag_list'] = ner_tag_list

ner_tag_list_max_len =list(map(lambda x :  ['O']+x + ( ['O']*(126-len(x)))+['O'],ner['ner_tag_list']))
ner['ner_tag_list_max_len_with_cls'] = ner_tag_list_max_len

def tag_to_var(x):
  return list(map(lambda k : labels_dict[k],x))

ner_tag_list_max_len_with_cls_to_var = list(map(lambda x : tag_to_var(x), ner['ner_tag_list_max_len_with_cls'] ))
ner['ner_tag_list_max_len_with_cls_to_var'] = ner_tag_list_max_len_with_cls_to_var
ner.drop(index=[76,10051,19817,47591],axis=0,inplace=True)
ner.head()


(47959, 3)


,sentences,tag_str,ner_tag_list,ner_tag_list_max_len_with_cls,ner_tag_list_max_len_with_cls_to_var
0,Thousands of demonstrators have marched throu...,",O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo...","[O, O, O, O, O, O, O, B-geo, O, O, O, O, O, B-...","[0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 0, ..."
1,Families of soldiers killed in the conflict j...,",O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-per,O,O...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,They marched from the Houses of Parliament to...,",O,O,O,O,O,O,O,O,O,O,O,B-geo,I-geo,O","[O, O, O, O, O, O, O, O, O, O, O, B-geo, I-geo...","[O, O, O, O, O, O, O, O, O, O, O, O, B-geo, I-...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 11, 0,..."
3,"Police put the number of marchers at 10,000 w...",",O,O,O,O,O,O,O,O,O,O,O,O,O,O,O","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,The protest comes on the eve of the annual co...,",O,O,O,O,O,O,O,O,O,O,O,B-geo,O,O,B-org,I-org,O...","[O, O, O, O, O, O, O, O, O, O, O, B-geo, O, O,...","[O, O, O, O, O, O, O, O, O, O, O, O, B-geo, O,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, ..."


In [6]:
#ner = ner.iloc[0:3000,:].copy()
print(ner.shape)

(47955, 5)


In [7]:
from transformers import AutoConfig, TFAutoModelForTokenClassification,AutoTokenizer

MODEL_NAME = 'bert-base-uncased' 

config = AutoConfig.from_pretrained(MODEL_NAME, num_labels=18)
#config.output_attentions = True
#config.output_hidden_states = True
#config.use_cache = True
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = TFAutoModelForTokenClassification.from_pretrained(MODEL_NAME, config=config)
model.summary()

All model checkpoint layers were used when initializing TFBertForTokenClassification.

Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_token_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108891648 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  13842     
Total params: 108,905,490
Trainable params: 108,905,490
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.layers

In [8]:
sen_list = list(map(lambda x: x.split(),ner['sentences']))

In [9]:
tag_words = ner['ner_tag_list']

In [10]:
def tag_to_var(x):
  return list(map(lambda k : labels_dict[k],x))

tag_var = list(map(lambda x : tag_to_var(x), tag_words ))


In [11]:
tokenized_inputs = tokenizer(sen_list, is_split_into_words=True,add_special_tokens=True,max_length=512,padding='max_length')
#tokenized_inputs


In [12]:
print(tokenized_inputs['input_ids'][0])
print(tokenized_inputs['attention_mask'][0])

[101, 5190, 1997, 28337, 2031, 9847, 2083, 2414, 2000, 6186, 1996, 2162, 1999, 5712, 1998, 5157, 1996, 10534, 1997, 2329, 3629, 2013, 2008, 2406, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [13]:
len(tokenized_inputs['input_ids'][0])


512

In [14]:
for i in range(0,3000):
  sen_len = len(ner.iloc[i,0].split())
  tag_len = len(ner.iloc[i,2])
  if sen_len != tag_len:
    print(i)

In [15]:
def aligned_the_labels(example,tokens):
  tokenized_example = tokenizer(example, truncation=True, is_split_into_words=True,add_special_tokens=True,max_length=512,padding='max_length')
  word_ids = tokenized_example.word_ids()
  aligned_labels = [17 if i is None else tokens[i] for i in word_ids]
  return aligned_labels

aligned_labels = list(map(lambda x,y :aligned_the_labels(x,y),sen_list,tag_var ))

In [16]:
type(aligned_labels[0][0])

int

In [17]:
np.array(aligned_labels).shape

(47955, 512)

In [18]:
# tf.data.Dataset.from_tensor_slices(tokenized_inputs['input_ids'])
 # tf.data.Dataset.from_tensor_slices(aligned_the_labels)
BATCH_SIZE=8
EPOCHS = 1
optimizer = tf.keras.optimizers.Adam(lr=0.00001) #lr=0.00001
#loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss = SparseCategoricalFocalLoss(gamma=2)
model.compile(optimizer=optimizer, loss=loss, metrics='accuracy')

history=model.fit(x= {'input_ids': tf.constant(tokenized_inputs['input_ids']),'token_type_ids':tf.constant(tokenized_inputs['token_type_ids']),
                      'attention_mask':tf.constant(tokenized_inputs['attention_mask'])},
                  y=tf.constant(aligned_labels),
                  #validation_split=0.2,
                  batch_size=BATCH_SIZE,epochs=EPOCHS)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [19]:
pred_train = []
k =1000
for i in range(0,k):
  a = model.predict({'input_ids': tf.constant(tokenized_inputs['input_ids'][i]),'token_type_ids':tf.constant(tokenized_inputs['token_type_ids'][i]),'attention_mask':tf.constant(tokenized_inputs['attention_mask'][i])})[0]
  a = np.transpose(a,(1,0,2))
  a = a[0]
  pred_train.append(list(map(lambda x : list(x).index(max(list(x))), a )))

print(classification_report( np.array(aligned_labels[0:k]).ravel()   ,  np.array(pred_train).ravel()    )) 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.45      0.05      0.09     19995
           1       0.00      0.00      0.00        68
           2       0.00      0.00      0.00        24
           3       0.04      0.01      0.02       854
           4       0.05      0.14      0.08       738
           5       0.00      0.00      0.00        19
           6       0.09      0.07      0.08       748
           7       0.01      0.03      0.02       494
           8       0.00      0.00      0.00       373
           9       0.00      0.00      0.00        26
          10       0.00      0.00      0.00        17
          11       0.01      0.01      0.01       139
          12       0.00      0.00      0.00        28
          13       0.00      0.00      0.00         5
          14       0.02      0.01      0.01       327
          15       0.00      0.00      0.00       774
          16       0.00      0.00      0.00        77
          17       0.96    

In [ ]:
# tf.data.Dataset.from_tensor_slices(tokenized_inputs['input_ids'])
 # tf.data.Dataset.from_tensor_slices(aligned_the_labels)


BATCH_SIZE=8
EPOCHS =10
optimizer = tf.keras.optimizers.Adam(lr=0.00001)
#loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss = SparseCategoricalFocalLoss(gamma=2)
model.compile(optimizer=optimizer, loss=loss, metrics='accuracy')

history=model.fit(x= {'input_ids': tf.constant(tokenized_inputs['input_ids']),'token_type_ids':tf.constant(tokenized_inputs['token_type_ids']),
                      'attention_mask':tf.constant(tokenized_inputs['attention_mask'])},
                  y=tf.constant(aligned_the_labels),
                  #validation_split=0.2,
                  batch_size=BATCH_SIZE,epochs=EPOCHS)

Epoch 1/10
375/375 [==============================] - 374s 979ms/step - loss: 0.0017 - accuracy: 0.9836
Epoch 2/10
375/375 [==============================] - 366s 976ms/step - loss: 6.2962e-04 - accuracy: 0.9870
Epoch 3/10
375/375 [==============================] - 366s 977ms/step - loss: 4.1378e-04 - accuracy: 0.9878
Epoch 4/10
375/375 [==============================] - 366s 976ms/step - loss: 3.1025e-04 - accuracy: 0.9880
Epoch 5/10
375/375 [==============================] - 366s 976ms/step - loss: 2.4418e-04 - accuracy: 0.9879
Epoch 6/10
375/375 [==============================] - 366s 976ms/step - loss: 1.9483e-04 - accuracy: 0.9867
Epoch 7/10
375/375 [==============================] - 366s 976ms/step - loss: 1.6742e-04 - accuracy: 0.9856
Epoch 8/10
375/375 [==============================] - 366s 976ms/step - loss: 1.4038e-04 - accuracy: 0.9850
Epoch 9/10
375/375 [==============================] - 366s 976ms/step - loss: 1.2956e-04 - accuracy: 0.9846
Epoch 10/10
375/375 [===========

In [ ]:
pred_train = []
k =100
for i in range(0,k):
  a = model.predict({'input_ids': tf.constant(tokenized_inputs['input_ids'][i]),'token_type_ids':tf.constant(tokenized_inputs['token_type_ids'][i]),'attention_mask':tf.constant(tokenized_inputs['attention_mask'][i])})[0]
  a = np.transpose(a,(1,0,2))
  a = a[0]
  pred_train.append(list(map(lambda x : list(x).index(max(list(x))), a )))

print(classification_report( np.array(aligned_the_labels[0:k]).ravel()   ,  np.array(pred_train).ravel()    )) 

              precision    recall  f1-score   support

           0       0.99      1.00      1.00     50740
           1       0.00      0.00      0.00         1
           3       0.00      0.00      0.00        95
           4       0.00      0.00      0.00        69
           6       0.00      0.00      0.00        97
           7       0.00      0.00      0.00        38
           8       0.00      0.00      0.00        29
           9       0.00      0.00      0.00         3
          11       0.00      0.00      0.00        26
          12       0.00      0.00      0.00         2
          14       0.00      0.00      0.00        51
          15       0.00      0.00      0.00        47
          16       0.00      0.00      0.00         2

    accuracy                           0.99     51200
   macro avg       0.08      0.08      0.08     51200
weighted avg       0.98      0.99      0.99     51200



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# tf.data.Dataset.from_tensor_slices(tokenized_inputs['input_ids'])
 # tf.data.Dataset.from_tensor_slices(aligned_the_labels)


BATCH_SIZE=8
EPOCHS =10
optimizer = tf.keras.optimizers.Adam(lr=0.00001)
#loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss = SparseCategoricalFocalLoss(gamma=2)
model.compile(optimizer=optimizer, loss=loss, metrics='accuracy')

history=model.fit(x= {'input_ids': tf.constant(tokenized_inputs['input_ids']),'token_type_ids':tf.constant(tokenized_inputs['token_type_ids']),
                      'attention_mask':tf.constant(tokenized_inputs['attention_mask'])},
                  y=tf.constant(aligned_the_labels),
                  #validation_split=0.2,
                  batch_size=BATCH_SIZE,epochs=EPOCHS)

Epoch 1/10
375/375 [==============================] - 373s 978ms/step - loss: 1.0253e-04 - accuracy: 0.9837
Epoch 2/10
375/375 [==============================] - 367s 978ms/step - loss: 9.6440e-05 - accuracy: 0.9834
Epoch 3/10
375/375 [==============================] - 366s 976ms/step - loss: 8.4352e-05 - accuracy: 0.9845
Epoch 4/10
375/375 [==============================] - 366s 976ms/step - loss: 8.5127e-05 - accuracy: 0.9831
Epoch 5/10
375/375 [==============================] - 367s 978ms/step - loss: 8.7321e-05 - accuracy: 0.9834
Epoch 6/10
375/375 [==============================] - 366s 977ms/step - loss: 6.9501e-05 - accuracy: 0.9838
Epoch 7/10
375/375 [==============================] - 366s 976ms/step - loss: 6.2598e-05 - accuracy: 0.9833
Epoch 8/10
375/375 [==============================] - 366s 977ms/step - loss: 6.1343e-05 - accuracy: 0.9844
Epoch 9/10
375/375 [==============================] - 367s 977ms/step - loss: 6.1591e-05 - accuracy: 0.9842
Epoch 10/10
375/375 [=======

In [ ]:
pred_train = []
k =1000
for i in range(0,k):
  a = model.predict({'input_ids': tf.constant(tokenized_inputs['input_ids'][i]),'token_type_ids':tf.constant(tokenized_inputs['token_type_ids'][i]),'attention_mask':tf.constant(tokenized_inputs['attention_mask'][i])})[0]
  a = np.transpose(a,(1,0,2))
  a = a[0]
  pred_train.append(list(map(lambda x : list(x).index(max(list(x))), a )))

print(classification_report( np.array(aligned_the_labels[0:k]).ravel()   ,  np.array(pred_train).ravel()    )) 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.99      1.00      1.00    507289
           1       0.00      0.00      0.00        68
           2       0.00      0.00      0.00        24
           3       0.00      0.00      0.00       854
           4       0.00      0.00      0.00       738
           5       0.00      0.00      0.00        19
           6       0.00      0.00      0.00       748
           7       0.00      0.00      0.00       494
           8       0.00      0.00      0.00       373
           9       0.00      0.00      0.00        26
          10       0.00      0.00      0.00        17
          11       0.00      0.00      0.00       139
          12       0.00      0.00      0.00        28
          13       0.00      0.00      0.00         5
          14       0.00      0.00      0.00       327
          15       0.00      0.00      0.00       774
          16       0.00      0.00      0.00        77

    accuracy              

dump

In [26]:
print(len(ainput_ids))
print(len(ainput_mask)   )
print(len(asegment_ids)  )
print(len(alabel_ids)    )
print(len(alabel_mask)  ) 

16
16
16
16
16


In [ ]:
#from transformers import AutoTokenizer, AutoModel,TFAutoModel
#tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT",from_pt=True)
#transformer_model = TFAutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT",from_pt=True)
#max_length = 128